### This file carries out orthogonal projection from a Cartesian point in space to an ellipsoid Earth.

Equatorial Radius $r_{e} =  6,378,136.6 $ $ (m)$

Polar Radius $r_{p} =   6,356,751.9 $ $ (m)$

Mathematical Expression of Earth in Cartesian coordinates:

$$ \frac{x^2}{{r_e}^2} + \frac{y^2}{{r_p}^2} + \frac{z^2}{{r_e}^2} = 1 $$

In [2]:
import numpy as np
import utilities as ut
import math
import numpy as np
from matplotlib import pyplot as plt
import utilities
from Earth import Earth
from ISimulator import ISimulator, FakeSimulator, RealSimulator
from RadarSystem import RadarSystem
from TwoD.TwoDRadarSystem import TwoDRadarSystem
from config import config
from TwoD.TwoDSimulator import TwoDSimulator



class Simulation():
    def __init__(self, satellite, planet, radars,
                 dt = 0.1, maxIterations = 1000000):
        
        #Satellite, Planet and Radars are objects
        self.sat = satellite
        self.Earth = planet
        self.radars = radars

        self.dt = dt
        self.maxIter = maxIterations
        self.Crashed_Flag = False
        #We use intger time.
        self.t = 0
        
        #These are the logging variables used for CSV (or any other) file gen.
        self.Satellite_True_Traj = []
        self.Satellite_Kalman_Traj = []
        self.Satellite_Kalman_Cov = []
        self.init_config()
        ''' TODO:
        This is where the config file would be read and its contents loaded into respective variables.
        '''
      

    def init_config(self):
        """
    Initializes the system configuration based on specified parameters.
    This method sets up the radar system and simulator based on the
    configuration flags is2D and real_simulator.
    
    Attributes:
        is2D (bool): If True, the system will use a two-dimensional
                     radar system and simulator. If False, it will use
                     a three-dimensional system.
        real_simulator (bool): If True, use the RealSimulator for
                               realistic simulations. If False, read existed data from file.

    
    This setup allows for easy swapping of system behavior by simply
    changing configuration flags, adhering to the Adapter Design Pattern.
    The rest of the program interacts with the radar system and simulator
    through interfaces, ensuring that changes in system configuration do not
    require changes elsewhere in the program.
    """
        is2D = True
        real_simulator = True
        if is2D:
            self.simulator = TwoDSimulator()
            self.radarSystem = TwoDRadarSystem(1, Earth())
        else:
            self.radarSystem = RadarSystem(1, Earth())
            if real_simulator:
                self.simulator = FakeSimulator()
            else:
                self.simulator = RealSimulator()


    def Fall(self):
        '''
        1. Declare intial conditions. -- DONE IN __init__()
        2. Enter the while loop until Crashed 
        3. Update the radar positons
        4. Update the TRUE satellite position through dynamics
        5. Radars LOS check
            5a. check for LOS for each, 
            5b. IF data update LKF update
            5c. Update Radar pings 
        6. Satellite LKF forecast  
        7. Increment simulation
        '''
        
        '''
        For the third step, I think using interface is a good idea.When we want to debug radars all debug predictors, simulate trajectories every time will waste time.So we can have two implementations of the same Interface(ISimulator),One is called FakeSimulator, which return exsiting data from storage, another one is the real simulator which update dynamics at each time step .By using interface,we can switch these two mode very quickly
        Following is an example
        '''
        
        current_time = 0
        while not self.Crashed_Flag:
            current_time = current_time + 1
            '''
            4.Update the TRUE satellite position through dynamics
            '''
            satellite_state = self.simulator.simulate(current_time)
            '''
            3. Update the radar positons
            5. Radars LOS check
                5a. check for LOS for each, 
                5b. IF data update LKF update
                5c. Update Radar pings 
            '''
            # radar has different time interval from simulator, so radar only works at longer periodic step
            if current_time % config['sim_config']['dt']['radar_los'] == 0:
                self.radarSystem.update_radar_positions()
                self.radarSystem.try_detect_satellite(satellite_state.pos, current_time)
 
        
        
            
        

    def get_time(self):
        return self.dt * self.t


ModuleNotFoundError: No module named 'SatelliteState'

In [7]:
a = np.zeros(2)

In [10]:
len(a)

2